In [1]:
import os
import time
import copy
import argparse
import numpy as np
import torch
import torch.nn as nn
from torchvision.utils import save_image
from utils import get_loops, get_dataset, get_network, get_eval_pool, evaluate_synset, get_daparam, match_loss, get_time, TensorDataset, epoch, DiffAugment, ParamDiffAug

In [2]:
ipc=10
num_classes=10
channel=3
im_size=(32,32)

In [3]:
label_syn = torch.tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7,
        7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9,
        9, 9, 9, 9,9, 9, 9, 9, 9, 9, 9, 9, 9, 9], dtype=torch.long, requires_grad=False, device=args.device).view(-1) # [0,0,0, 1,1,1, ..., 9,9,9]

/tmp/ipykernel_11325/3843153445.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400430266/work/torch/csrc/utils/tensor_new.cpp:261.)
  label_syn = torch.tensor([np.ones(ipc)*i for i in range(num_classes)], dtype=torch.long, requires_grad=False).view(-1) # [0,0,0, 1,1,1, ..., 9,9,9]


In [22]:
image_syn = torch.randn(size=(num_classes*13, channel, im_size[0], im_size[1]), dtype=torch.float, requires_grad=True)

In [12]:
img_syn = image_syn[10:30].reshape((20, channel, im_size[0], im_size[1]))

In [31]:
id_1=0
for c in range(num_classes):
    
    if c in [1,2,9]:
        id_1 = id_1 + 20
        img_syn = image_syn[id_1-20:id_1].reshape((20, channel, im_size[0], im_size[1]))
        lab_syn = torch.ones((10 + 10,), dtype=torch.long) * c
    else:
        id_1 = id_1 + 10
        img_syn = image_syn[id_1-10:id_1].reshape((10, channel, im_size[0], im_size[1]))
        lab_syn = torch.ones((10,),  dtype=torch.long) * c